---
title: Tracking Changes
date: 2023-11-30 
authors:
  - name: Sébastien Boisgérault
    email: Sebastien.Boisgerault@minesparis.psl.eu
    url: https://github.com/boisgera
    affiliations:
      - institution: Mines Paris - PSL University
        department: Institut des Transformation Numériques (ITN)
github: boisgera
license: CC-BY-4.0
open_access: true
---

In order to understand how `.tldr` files are structured, we can add a new graphical objects, change some if their properties, etc. and each time we modify the document, analyze the corresponding evolution of the file.

In this notebook, we develop some tooling to help us track such changes.

## Text comparison

We define two similar versions of the "zen of Python":

In [2]:
zen_1 = """The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Errors should never pass silently.
In the face of ambiguity, refuse the temptation to guess.
There should be one obvious way to do it.
Although that way may not be obvious at first.
Now is better than never.
Although never is often better than right now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it is a good idea.
"""

In [3]:
zen_2 = """\
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
"""

```{exercise}
 1. Transform `zen_1` and `zen_2` into list of lines.
 2. Use the [`difflib`](https://docs.python.org/3/library/difflib.html) module of the Python standard library to [`compare`](https://docs.python.org/3/library/difflib.html#difflib.Differ.compare) the two sequences.
 3. Make a text out of the output of compare and print it.
 4. Interpret the result and list the differences between both versions of the zen of Python.
```

In [4]:
list_lines_1 = zen_1.splitlines()
list_lines_2 = zen_2.splitlines()

In [5]:
list_lines_1

['The Zen of Python, by Tim Peters',
 '',
 'Beautiful is better than ugly.',
 'Explicit is better than implicit.',
 'Simple is better than complex.',
 'Complex is better than complicated.',
 'Flat is better than nested.',
 'Sparse is better than dense.',
 'Readability counts.',
 "Special cases aren't special enough to break the rules.",
 'Errors should never pass silently.',
 'In the face of ambiguity, refuse the temptation to guess.',
 'There should be one obvious way to do it.',
 'Although that way may not be obvious at first.',
 'Now is better than never.',
 'Although never is often better than right now.',
 "If the implementation is hard to explain, it's a bad idea.",
 'If the implementation is easy to explain, it is a good idea.']

In [6]:
import difflib as diff

differ = diff.Differ()  #on appelle la classe Differ du module difflib

comparison = differ.compare(list_lines_1, list_lines_2)  #on utilise la méthode compare de la classe Differ
comparison_list = list(comparison) #on convertit le générateur en une liste

for line in comparison_list :
   print(line)


  The Zen of Python, by Tim Peters
  
  Beautiful is better than ugly.
  Explicit is better than implicit.
  Simple is better than complex.
  Complex is better than complicated.
  Flat is better than nested.
  Sparse is better than dense.
  Readability counts.
  Special cases aren't special enough to break the rules.
+ Although practicality beats purity.
  Errors should never pass silently.
+ Unless explicitly silenced.
  In the face of ambiguity, refuse the temptation to guess.
- There should be one obvious way to do it.
+ There should be one-- and preferably only one --obvious way to do it.
- Although that way may not be obvious at first.
+ Although that way may not be obvious at first unless you're Dutch.
?                                              ++++++++++++++++++++

  Now is better than never.
- Although never is often better than right now.
+ Although never is often better than *right* now.
?                                     +     +

  If the implementation is hard to exp

We can make our job easier if we use HTML instead of plain text to visualise the differences between the two texts.


```{exercise}
  1. Use the [HtmlDiff](https://docs.python.org/3/library/difflib.html#difflib.HtmlDiff) class of difflib to produce a `diff.html` file that represents this difference in a HTML document.
  2. Use the [webbrowser](https://docs.python.org/3/library/webbrowser.html) module of the standard library to open it!
  3. Define a `display_diff_text` function that takes two arguments `text_1` and `text_2` and automates steps 1. and 2.
```

In [7]:
import webbrowser

html_diff = diff.HtmlDiff()

sortie_html = html_diff.make_file(list_lines_1, list_lines_2)
/7
file = open("diff.html", "w")
file.write(sortie_html)

webbrowser.open("diff.html")  # problème je n'arrive pas à ouvrir le fichier dans une fenêtre.

True

In [8]:
from IPython.display import HTML

def display_diff_text (text_1, text_2) :
    """reçoit en argument deux textes, renvoie un fichier html contenant les différences entre les deux textes"""
    list_lines_1 = text_1.splitlines()
    list_lines_2 = text_2.splitlines()
    sortie_html = html_diff.make_file(list_lines_1, list_lines_2)
    name_1 = str(text_1)
    name_2 = str(text_2)
    name = f"{name_1}{name_2}diff.html"
    file = open("name", "w")
    file.write(sortie_html)
    return HTML("name")

In [9]:
display_diff_text (zen_1, zen_2)

f,1,"The Zen of Python, by Tim Peters",f,1,"The Zen of Python, by Tim Peters"
,2,,,2,
,3,Beautiful is better than ugly.,,3,Beautiful is better than ugly.
,4,Explicit is better than implicit.,,4,Explicit is better than implicit.
,5,Simple is better than complex.,,5,Simple is better than complex.
,6,Complex is better than complicated.,,6,Complex is better than complicated.
,7,Flat is better than nested.,,7,Flat is better than nested.
,8,Sparse is better than dense.,,8,Sparse is better than dense.
,9,Readability counts.,,9,Readability counts.
,10,Special cases aren't special enough to break the rules.,,10,Special cases aren't special enough to break the rules.
n,,,n,11,Although practicality beats purity.


## Comparison of JSON documents

````{exercise} Comparison of dictionnaries

 1. Create a `display_diff` function that takes two Python objects, converts them to strings then leverages `display_diff_text` to display the difference in a browser.

 2. Consider the 3 dictionaries defined by
    ```python
    d1 = {k:k+1 for k in range(100)}
    d2 = d1.copy(); d2[50] = 50
    d3 = {k:k+1 for k in range(99, -1, -1)}
    ```
    `d1` and `d2` have a slight difference and `d1` and `d2` are equal.
    Does your `display_diff` function make easy to spot where the difference is in the first case when it compares `d1` and `d2`?
    Does it make easy to see that `d1` and `d3` are equal?

  3. Investigate the [`pprint`](https://docs.python.org/3/library/pprint.html) module standard library ; use it to improve the behavior of `display_text_diff` in the two cases considered in the previous question.

````
 

In [10]:
def display_diff (object1, object2) :
    str_1 = str(object1)
    str_2 = str(object2)
    return display_diff_text(str_1, str_2)

In [11]:
d1 = {k:k+1 for k in range(100)}
d2 = d1.copy(); d2[50] = 50
d3 = {k:k+1 for k in range(99, -1, -1)}

In [17]:
ls1 = list(zen_1)
ls2 = list(zen_2)

display_diff(ls1, ls2)

In [16]:
import pprint
pp = pprint.PrettyPrinter(width=41, compact=True, stream = None)

pp.pprint(d1)

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 44,
 44: 45,
 45: 46,
 46: 47,
 47: 48,
 48: 49,
 49: 50,
 50: 51,
 51: 52,
 52: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 65: 66,
 66: 67,
 67: 68,
 68: 69,
 69: 70,
 70: 71,
 71: 72,
 72: 73,
 73: 74,
 74: 75,
 75: 76,
 76: 77,
 77: 78,
 78: 79,
 79: 80,
 80: 81,
 81: 82,
 82: 83,
 83: 84,
 84: 85,
 85: 86,
 86: 87,
 87: 88,
 88: 89,
 89: 90,
 90: 91,
 91: 92,
 92: 93,
 93: 94,
 94: 95,
 95: 96,
 96: 97,
 97: 98,
 98: 99,
 99: 100}


```{exercise} tldraw documents comparator
Implement a function `tldraw_diff` that takes as argument two filenames that refer to tldraw documents and display their differences in the browser.
```

In [21]:
def tldraw_diff (filename_1, filename_2) :
    file_1 = open(filename_1, mode="tr", encoding = "utf-8")
    file_2 = open(filename_2, mode="tr", encoding = "utf-8")
    content_1 = file_1.read() #dictionnaire
    content_2 = file_2.read() #dictionnaire
    return display_diff(content_1, content_2)

In [23]:
tldraw_diff("hello-python.tldr", "hello-tldraw.tldr")